In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
from tqdm import tqdm

import os
import torch
import warnings
import numpy as np

from generate_instances import generic

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
root = 'datasets/qp_generic_300_0.01_0.01'
os.mkdir(root)
os.mkdir(os.path.join(root, 'processed'))

### generic

In [ ]:
A_density = 0.01
P_density = 0.01
nrows = 300
ncols = 300

def surrogate_gen():
    A, b, P, q, success = generic(nrows, ncols, A_density, P_density, rng)
    return A, b, P, q, success

# create QP

In [ ]:
from solver.linprog_ip import _ip_hsd_feas
from scipy.linalg import null_space
from torch_geometric.data import Batch, HeteroData, InMemoryDataset
from qpsolvers import solve_qp

In [ ]:
graphs = []
pkg_idx = 0
success_cnt = 0

max_iter = 12000
num = 10000

for i in tqdm(range(max_iter)):
    try:
        A, b, P, q, success = surrogate_gen()
        assert success

        G = None
        h = None

        nulls = null_space(A)
        solution = solve_qp(P, q, G, h, A.astype(np.float64), b.astype(np.float64), 
                            lb=np.zeros(A.shape[1]).astype(np.float64), solver="cvxopt")
        assert solution is not None
        obj = 0.5 * solution @ P @ solution + q.dot(solution)
        assert not np.isnan(obj)
        x_feasible, *_ = _ip_hsd_feas(A, b, np.zeros(A.shape[1]), 0.,
                                      alpha0=0.99995, beta=0.1,
                                      maxiter=100, tol=1.e-6, sparse=False,
                                      lstsq=False, sym_pos=True, cholesky=None,
                                      pc=True, ip=True, permc_spec='MMD_AT_PLUS_A',
                                      rand_start=True)
        assert np.all(x_feasible >= 0.) and np.abs(A @ x_feasible - b).max() < 1.e-6
    except AssertionError:
        continue
    else:        
        A = torch.from_numpy(A).to(torch.float)
        b = torch.from_numpy(b).to(torch.float)
        q = torch.from_numpy(q).to(torch.float)
        solution = torch.from_numpy(solution).to(torch.float)
        x_feasible = torch.from_numpy(x_feasible).to(torch.float)

        # todo: other decomposition methods
        # S = torch.from_numpy(P ** 0.5).to(torch.float)

        P = torch.from_numpy(P).to(torch.float)
        A_where = torch.where(A)
        P_where = torch.where(P)

        data = HeteroData(
            cons={
                'num_nodes': A.shape[0],
                'x': torch.empty(A.shape[0]),
                 },
            vals={
                'num_nodes': A.shape[1],
                'x': torch.empty(A.shape[1]),
            },
            cons__to__vals={'edge_index': torch.vstack(A_where),
                            'edge_attr': A[A_where][:, None]},
            vals__to__vals={'edge_index': torch.vstack(P_where),
                            'edge_attr': P[P_where][:, None]},
            x_solution=solution,
            x_feasible=x_feasible,
            obj_solution=obj,
            b=b,
            q=q,
            nulls=torch.from_numpy(nulls).float().reshape(-1)
        )
        success_cnt += 1
        graphs.append(data)

    if len(graphs) >= 1000 or success_cnt == num:
        torch.save(Batch.from_data_list(graphs), f'{root}/processed/batch{pkg_idx}.pt')
        pkg_idx += 1
        graphs = []

    if success_cnt >= num:
        break

In [ ]:
from data.dataset import LPDataset

In [ ]:
ds = LPDataset(root)